In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-12-09"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
16221,2024-12-09,Brasil Nbb,19:30,Fortaleza B.C.,São José,1.97,1.71,148.5,1.86,1.80,1.5,1.84,1.82,IBuMQw5M,0.507614,0.584795,0.537634,0.555556,0.092410,0.2,0.6,NaN,NaN,222.344,130.443312,0.586673,222.846,200.397315,0.899264,168.304,287.576,101.84,132.82,0.0,0.0,0.0,0.0,0.099917,0.023184,0.007728,-1.32,-0.264,-3.674242,0.603217,0.5,-0.103217,1.75,0.350,2.028571,0.379537,0.3,-0.079537
16222,2024-12-09,Brasil Nbb,19:30,Unifacisa,Mogi,1.18,4.10,152.5,1.83,1.83,-11.5,1.95,1.72,EsRiLHSq,0.847458,0.243902,0.546448,0.546448,0.091360,0.8,0.2,NaN,NaN,177.884,82.475827,0.463649,260.082,159.820214,0.614499,140.262,326.874,106.40,198.03,0.0,0.0,0.0,0.0,0.782103,0.000000,0.088629,-2.28,-0.456,-0.394737,0.700484,0.4,-0.300484,-5.00,-1.000,-3.100000,0.236198,0.1,-0.136198
16223,2024-12-09,Brasil Nbb,20:00,R10 Score Vasco da Gama,Minas,2.95,1.33,150.5,1.80,1.86,5.5,2.02,1.67,xGCnsak3,0.338983,0.751880,0.555556,0.537634,0.090863,0.6,0.4,NaN,NaN,166.558,58.096348,0.348806,127.806,48.514710,0.379596,125.098,133.438,127.19,104.50,0.0,0.0,0.0,0.0,0.535286,0.023184,0.134139,-2.08,-0.416,-4.687500,0.681572,0.7,0.018428,2.88,0.576,0.572917,0.696208,0.9,0.203792
16224,2024-12-09,Alemanha Copa Da Alemanha,16:00,Bayern,Vechta,1.09,6.69,163.5,1.80,1.86,-14.5,2.00,1.72,hIBzkFrf,0.917431,0.149477,0.555556,0.537634,0.066908,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,236.74,0.0,0.0,0.0,0.0,1.017943,0.023184,0.106446,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
16225,2024-12-09,Arábia Saudita Premier League,10:00,Al-Fateh,Ohod,1.21,3.74,171.5,1.83,1.83,-10.5,1.93,1.74,GdQvyBYb,0.826446,0.267380,0.546448,0.546448,0.093826,0.4,0.0,NaN,NaN,646.148,549.962334,0.851140,718.776,607.209309,0.844782,113.400,267.808,95.16,730.80,0.0,1.0,0.0,0.0,0.722820,0.000000,0.073215,-1.37,-0.274,-0.766423,0.000000,0.0,0.000000,-3.98,-0.796,-3.442211,0.000000,0.0,0.000000
16226,2024-12-09,Argentina Liga A,22:00,Olimpico,Ferro,1.11,5.17,160.5,1.83,1.83,-12.5,1.98,1.70,EarTvstm,0.900901,0.193424,0.546448,0.546448,0.094324,0.8,0.4,NaN,NaN,134.110,51.808433,0.386313,306.458,206.320480,0.673242,168.730,301.892,210.60,642.40,0.0,1.0,0.0,0.0,0.914285,0.000000,0.107603,4.63,0.926,0.118790,0.734545,0.9,0.165455,-2.66,-0.532,-7.838346,0.360175,0.4,0.039825
16227,2024-12-09,Ásia Wasl,12:00,Al Ittihad,Al Manama,1.72,1.95,170.5,1.83,1.83,-2.5,1.95,1.72,40ehmSx2,0.581395,0.512821,0.546448,0.546448,0.094216,0.6,0.0,NaN,NaN,598.846,1046.761323,1.747964,124.864,27.625229,0.221243,146.788,0.000,145.14,127.40,0.0,0.0,0.0,0.0,0.088629,0.000000,0.088629,3.28,0.656,1.097561,0.564934,0.7,0.135066,0.00,0.000,inf,0.000000,0.0,0.000000
16228,2024-12-09,Bósnia E Herzegovina Prvenstvo Bih Feminina,15:45,Feniks F,Mladi Krajisnik F,1.83,1.83,130.5,1.83,1.83,25.5,1.84,1.82,4CJrVAAT,0.546448,0.546448,0.546448,0.546448,0.092896,0.0,0.4,NaN,NaN,NaN,NaN,NaN,172.306,91.506902,0.531072,0.000,277.216,84.18,218.42,0.0,0.0,0.0,0.0,0.000000,0.000000,0.007728,0.00,0.000,inf,0.000000,0.0,0.000000,1.10,0.220,3.772727,0.000000,0.0,0.000000
16229,2024-12-09,Bulgária Nbl,14:15,Beroe,Cherno More,1.70,1.99,166.5,1.83,1.83,-3.5,2.05,1.65,r1amJwRH,0.588235,0.502513,0.546448,0.546448,0.090748,0.8,0.4,NaN,NaN,209.880,85.241955,0.406146,233.596,208.580833,0.892913,179.574,315.680,234.30,598.40,0.0,1.0,0.0,0.0,0.111144,0.000000,0.152888,-2.23,-0.446,-1.569507,0.499070,0.5,0.000930,-3.47,-0.694,-1.426513,0.366074,0.3,-0.06

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,21:00,Eua Ncaa,VCU Rams,Pennsylvania,133.5,1.91,0.8505,Over
1,21:30,Eua Nba,Toronto Raptors,New York Knicks,230.5,1.86,0.8998,Over
